In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext

from functools import reduce 
from pyspark.sql import DataFrame

import pyspark.sql.functions as f


conf = SparkConf()\
        .setAppName("pjt1")\
        .set("spark.driver.extraClassPath", "/opt/spark/jars/mysql-connector-java-8.0.26.jar")
sc = SparkContext(conf=conf)

sqlCtx = SQLContext(sc)
spark = sqlCtx.sparkSession

sql_url = "localhost"
user = "root"
password = "1234"
database = "proj"


sub2019 = spark.read.format("jdbc")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("url", "jdbc:mysql://{}:3306/{}?serverTimezone=Asia/Seoul ".format(sql_url, database))\
                .option("user", user)\
                .option("password", password)\
                .option("dbtable", 'sub2019')\
                .load()

sub2020 = spark.read.format("jdbc")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("url", "jdbc:mysql://{}:3306/{}?serverTimezone=Asia/Seoul ".format(sql_url, database))\
                .option("user", user)\
                .option("password", password)\
                .option("dbtable", 'sub2020')\
                .load()

sub2021 = spark.read.format("jdbc")\
                .option("driver", "com.mysql.jdbc.Driver")\
                .option("url", "jdbc:mysql://{}:3306/{}?serverTimezone=Asia/Seoul ".format(sql_url, database))\
                .option("user", user)\
                .option("password", password)\
                .option("dbtable", 'sub2021')\
                .load()

sub2019 = sub2019.drop('24over')
sub2020 = sub2020.drop('24over')

def unionAll(*dfs):
    return reduce(DataFrame.union, dfs)

sub_tot = unionAll(sub2019, sub2020, sub2021)

dateFormat = "yyyy-MM-dd"
sub_tot = sub_tot.withColumn("date",f.to_date(f.unix_timestamp(sub_tot.date, dateFormat).cast('timestamp')))

sub_tot.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/proj") \
    .option("dbtable", "proj.sub_total") \
    .option("user", "root") \
    .option("password", "1234") \
    .save()

---